In [1]:
import pandas as pd
import numpy as np
import preprocess as pp
import matplotlib.pyplot as plt
import datetime
from sklearn.linear_model import LinearRegression
from sklearn.decomposition import PCA
from sklearn.metrics import r2_score
import eda
import statsmodels.api as sm

In [2]:
def wls_results_wald(feature_df, target_df, x=None, z=None, y='y2'):

    q1 = feature_df[z].quantile(.25)
    q2 = feature_df[z].quantile(.5)
    q3 = feature_df[z].quantile(.75)
    
    I_q2 = np.where((feature_df[z] >= q1) & (feature_df[z] < q2), 1, 0)
    I_q3 = np.where((feature_df[z] >= q2) & (feature_df[z] < q3), 1, 0)
    I_q4 = np.where(feature_df[z] >= q3, 1, 0)
    
    x_ = np.array(feature_df[x]).reshape(-1,1)
    z_ = np.array(feature_df[z]).reshape(-1,1)
    
    x_z_q2 = I_q2.reshape(-1,1) * x_ * z_
    x_z_q3 = I_q3.reshape(-1,1) * x_ * z_
    x_z_q4 = I_q4.reshape(-1,1) * x_ * z_

    bias = np.ones(x_.shape)
    
    x__ = np.concatenate((bias, x_, x_z_q2, x_z_q3, x_z_q4), axis=1)
    y_ = target_df[y]
    weights = np.array(feature_df['weights']).reshape(-1,1)

    results = sm.WLS(y_, x__, weights=weights).fit()
    w = results.wald_test(np.eye(len(results.params))[2:5])
    f = w.fvalue
    p = w.pvalue
    
    summary = results.summary()
    f_pvalue = results.f_pvalue
    
    return f, p, summary, f_pvalue

In [3]:
def ranked_from_wald0(features, targets):
    z_list = features.loc[:,'z1':'z12'].columns
    x_list = features.loc[:,'x1':'x34'].columns

    f_list = []
    p_list = []
    for z in z_list:
        flist = []
        plist = []
        for feat in x_list:
            f0, p0 = wls_results_wald(features, targets, x=feat, z=z)
            flist.append(f0)
            plist.append(p0)

        f_list.append(flist)
        p_list.append(plist)
    
    f_dict = {'z1':f_list[0], 'z2':f_list[1], 'z3':f_list[2], 'z4':f_list[3],
              'z5':f_list[4], 'z6':f_list[5], 'z7':f_list[6], 'z8':f_list[7],
              'z9':f_list[8], 'z10':f_list[9], 'z11':f_list[10], 'z12':f_list[11]}

    f_df = pd.DataFrame.from_dict(f_dict, orient='index',
                           columns=x_list)

    p_dict = {'z1':p_list[0], 'z2':p_list[1], 'z3':p_list[2], 'z4':p_list[3],
              'z5':p_list[4], 'z6':p_list[5], 'z7':p_list[6], 'z8':p_list[7],
              'z9':p_list[8], 'z10':p_list[9], 'z11':p_list[10], 'z12':p_list[11]}

    p_df = pd.DataFrame.from_dict(p_dict, orient='index',
                           columns=x_list)
    
     
    p_list = []
    for col in p_df.columns:
        for idx in p_df.index:
            val = p_df.loc[idx, col]
            p_list.append(((col, idx), val))
    
    # ranked pairs with pvals
    ranked_p = eda.sort_scores1(p_list)
   
    # change K 
    return ranked_p[:10]

In [3]:
def normalize_and_fill(df):
    mean = df.loc[:,'x1':'z12'].mean()
    std = df.loc[:,'x1':'z12'].std()
    df.loc[:,'x1':'z12'] = (df.loc[:,'x1':'z12'] - mean)/std
    
    df = df.fillna(0)
    
    return df, mean, std

In [4]:
def train_and_test(features_df, targets_df, year=None, 
                   train_month_start=None, train_day_start=None,
                   train_month_end=None, train_day_end=None,
                   test_month_start=None, test_day_start=None,
                   test_month_end=None, test_day_end=None):
    
    train_features = features_df.loc[(features_df['datetime'].dt.date >= datetime.date(year, train_month_start, train_day_start))
                                    & (features_df['datetime'].dt.date <= datetime.date(year, train_month_end, train_day_end))]
    train_targets = targets_df.loc[(targets_df['datetime'].dt.date >= datetime.date(year, train_month_start, train_day_start))
                                    & (targets_df['datetime'].dt.date <= datetime.date(year, train_month_end, train_day_end))]
    
    test_features = features_df.loc[(features_df['datetime'].dt.date >= datetime.date(year, test_month_start, test_day_start))
                                    & (features_df['datetime'].dt.date <= datetime.date(year, test_month_end, test_day_end))]
    test_targets = targets_df.loc[(targets_df['datetime'].dt.date >= datetime.date(year, test_month_start, test_day_start))
                                    & (targets_df['datetime'].dt.date <= datetime.date(year, test_month_end, test_day_end))]
    
    return train_features, train_targets, test_features, test_targets

In [5]:
def selection(train_features, train_targets, test_features, test_targets):
    
    z_list = train_features.loc[:,'z1':'z12'].columns
    x_list = train_features.loc[:,'x1':'x34'].columns
    
    train_features, mean, std = normalize_and_fill(train_features)
    test_features.loc[:,'x1':] = (test_features.loc[:,'x1':]-mean)/std
    test_features = test_features.fillna(0)
    
    ranked_p = ranked_from_wald0(train_features, train_targets)
    
    pairs_list_train_top = []
    pairs_list_test_top = []
    pairs_list = []
    for pair in ranked_p:
        print(pair)
        pairs_list.append(pair[0])
        x = pair[0][0]
        z = pair[0][1]
        x_col_train = np.array(train_features[x]).reshape(-1,1)
        z_col_train = np.array(train_features[z]).reshape(-1,1)
        x_z_train = x_col_train * z_col_train
        pairs_list_train_top.append(x_z_train)
        x_col_test = np.array(test_features[x]).reshape(-1,1)
        z_col_test = np.array(test_features[z]).reshape(-1,1)
        x_z_test = x_col_test * z_col_test
        pairs_list_test_top.append(x_z_test)
        
    pairs_list_train_all = []
    pairs_list_test_all = []
    for x in x_list:
        x_col_train = np.array(train_features[x]).reshape(-1,1)
        x_col_test = np.array(test_features[x]).reshape(-1,1)
        pairs_list_train_all.append(x_col_train)
        pairs_list_test_all.append(x_col_test)
        for z in z_list:
            z_col_train = np.array(train_features[z]).reshape(-1,1)
            x_z_train = x_col_train * z_col_train
            pairs_list_train_all.append(x_z_train)
            z_col_test = np.array(test_features[z]).reshape(-1,1)
            x_z_test = x_col_test * z_col_test
            pairs_list_test_all.append(x_z_test)
        
    train_features_all = np.concatenate(pairs_list_train_all, axis=1)
    test_features_all = np.concatenate(pairs_list_test_all, axis=1)
    train_features_top = np.concatenate(pairs_list_train_top, axis=1)
    test_features_top = np.concatenate(pairs_list_test_top, axis=1)
    
    df = pd.DataFrame(np.array(pairs_list))
    
    return train_features_all, test_features_all, train_features_top, test_features_top, df

In [6]:
def cov_and_r2(train_features_all, train_features_top, train_targets, test_features_all, test_features_top, test_targets):
    
    pca = PCA(n_components=10)
    pca.fit(train_features_all)
    train_features_pca = pca.transform(train_features_all)
    test_features_pca = pca.transform(test_features_all)
    
    lr_all = LinearRegression().fit(train_features_all, np.array(train_targets['y2']).reshape(-1,1))
    lr_top = LinearRegression().fit(train_features_top, np.array(train_targets['y2']).reshape(-1,1))
    lr_pca = LinearRegression().fit(train_features_pca, np.array(train_targets['y2']).reshape(-1,1))
    
    pred_all = np.array(lr_all.predict(test_features_all)).reshape(-1,)
    pred_top = np.array(lr_top.predict(test_features_top)).reshape(-1,)
    pred_pca = np.array(lr_pca.predict(test_features_pca)).reshape(-1,)
    
    x = np.stack((pred_all, pred_top, pred_pca), axis=0)
    cov = np.cov(x)
    
    print('R2 from all features')
    print(r2_score(np.array(test_targets['y2']).reshape(-1,1), pred_all))
    print('R2 from top features')
    print(r2_score(np.array(test_targets['y2']).reshape(-1,1), pred_top))
    print('R2 from pca features')
    print(r2_score(np.array(test_targets['y2']).reshape(-1,1), pred_pca))
    
    return cov

In [8]:
jan_features = pp.read_npy1('/u/project/cratsch/tescala/month_split_right/features_jan_2015.npy', features=True)
jan_targets = pp.read_npy1('/u/project/cratsch/tescala/month_split_right/targets_jan_2015.npy', targets=True)

feb_features = pp.read_npy1('/u/project/cratsch/tescala/month_split_right/features_feb_2015.npy', features=True)
feb_targets = pp.read_npy1('/u/project/cratsch/tescala/month_split_right/targets_feb_2015.npy', targets=True)

mar_features = pp.read_npy1('/u/project/cratsch/tescala/month_split_right/features_mar_2015.npy', features=True)
mar_targets = pp.read_npy1('/u/project/cratsch/tescala/month_split_right/targets_mar_2015.npy', targets=True)

z_list = ['z2', 'z3', 'z4', 'z5', 'z6', 'z7', 'z8', 'z9', 'z10', 'z11', 'z12']

new_jan_features = jan_features.loc[:, 'datetime':'z1']
for z in z_list:
    new_jan_features[z] = jan_features[z]
    
new_feb_features = feb_features.loc[:, 'datetime':'z1']
for z in z_list:
    new_feb_features[z] = feb_features[z]
    
new_mar_features = mar_features.loc[:, 'datetime':'z1']
for z in z_list:
    new_mar_features[z] = mar_features[z]
    
comb_features = pd.concat([new_jan_features, new_feb_features, new_mar_features], ignore_index=True)
comb_targets = pd.concat([jan_targets, feb_targets, mar_targets], ignore_index=True)

In [9]:
train_features, train_targets, test_features, test_targets = train_and_test(comb_features, comb_targets, year=2015, 
                   train_month_start=1, train_day_start=5,
                   train_month_end=1, train_day_end=30,
                   test_month_start=2, test_day_start=2,
                   test_month_end=2, test_day_end=6)

In [26]:
train_features_all, test_features_all, train_features_top, test_features_top, df = selection(train_features, train_targets, test_features, test_targets)
df


/u/local/apps/anaconda3/2020.11/lib/python3.8/site-packages/pandas/core/indexing.py:1717: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, v)


(('x32', 'z7'), array(1.96997619e-175))
(('x7', 'z8'), array(2.23257168e-151))
(('x33', 'z7'), array(1.47841329e-144))
(('x2', 'z8'), array(5.46852246e-132))
(('x8', 'z8'), array(2.27825442e-130))
(('x8', 'z4'), array(1.42186772e-129))
(('x3', 'z8'), array(3.188399e-121))
(('x6', 'z8'), array(5.245387e-111))
(('x34', 'z7'), array(3.14470793e-110))
(('x5', 'z7'), array(1.16224417e-85))


,0,1
0,x32,z7
1,x7,z8
2,x33,z7
3,x2,z8
4,x8,z8
5,x8,z4
6,x3,z8
7,x6,z8
8,x34,z7
9,x5,z7


In [27]:
df

,0,1
0,x32,z7
1,x7,z8
2,x33,z7
3,x2,z8
4,x8,z8
5,x8,z4
6,x3,z8
7,x6,z8
8,x34,z7
9,x5,z7


In [11]:
cov = cov_and_r2(train_features_all, train_features_top, train_targets, test_features_all, test_features_top, test_targets)
cov

R2 from all features
0.002191605368637828
R2 from top features
0.0006632985064024544
R2 from pca features
0.0003889630826435786


array([[2.69763101e-08, 5.58738119e-09, 2.48590069e-09],
       [5.58738119e-09, 5.90822340e-09, 2.02523364e-09],
       [2.48590069e-09, 2.02523364e-09, 2.87053386e-09]])

In [ ]:
df.to_csv('top_feats_jan1_for_mob.csv', index=False)

In [39]:
train_month_start = [1,1,1,1,2,2,2,2]
train_day_start = [5,12,19,26,2,9,16,23]
train_month_end = [1,2,2,2,2,3,3,3]
train_day_end = [30,6,13,20,27,6,13,20]
test_month_start = [2,2,2,2,3,3,3,3]
test_day_start = [2,9,16,23,2,9,16,23]
test_month_end = [2,2,2,2,3,3,3,3]
test_day_end = [6,13,20,27,6,13,20,27]



for tms, tds, tme, tde, testms, testds, testme, testde in zip(train_month_start, train_day_start, train_month_end, train_day_end, test_month_start, test_day_start, test_month_end, test_day_end)
    
    train_features, train_targets, test_features, test_targets = train_and_test(comb_features, comb_targets, year=2015, 
                   train_month_start=tms, train_day_start=tds,
                   train_month_end=tme, train_day_end=tde,
                   test_month_start=testms, test_day_start=testds,
                   test_month_end=testme, test_day_end=testde)
    
    train_features_all, test_features_all, train_features_top, test_features_top, df = selection(train_features, train_targets, test_features, test_targets)

    cov = cov_and_r2(train_features_all, train_features_top, train_targets, test_features_all, test_features_top, test_targets)

    df.to_csv('topfeats_{}_{}_early_{}.csv'.format(tms, tds, 50) , index=False)

SyntaxError: invalid syntax (<ipython-input-39-2d60f9e2a5d7>, line 12)

In [ ]:
# Q2
train_month_start = [4,4,4,4,5,5,5,5]
train_day_start = [6,13,20,27,4,11,18,25]
train_month_end = [5,5,5,5,5,6,6,6]
train_day_end = [1,8,15,22,29,5,12,19]
test_month_start = [5,5,5,5,6,6,6,6]
test_day_start = [4,11,18,25,1,8,15,22]
test_month_end = [5,5,5,5,6,6,6,6]
test_day_end = [8,15,22,29,5,12,19,26]

# Q3
train_month_start = [7,7,7,7,8,8,8,8]
train_day_start = [6,13,20,27,3,10,17,24]
train_month_end = [7,8,8,8,8,9,9,9]
train_day_end = [31,7,14,21,28,4,11,18]
test_month_start = [8,8,8,8,9,9,9,9]
test_day_start = [3,10,17,24,31,7,14,21]
test_month_end = [8,8,8,8,9,9,9,9]
test_day_end = [7,14,21,28,4,11,18,25]

# Q4
train_month_start = [10,10,10,10,11,11,11,11]
train_day_start = [5,12,19,26,2,9,16,23]
train_month_end = [10,11,11,11,11,11,12,12]
train_day_end = [30,6,13,20,27,4,11,18]
test_month_start = [11,11,11,11,12,12,12,12]
test_day_start = [2,9,16,23,30,7,14,21]
test_month_end = [11,11,11,11,12,12,12,12]
test_day_end = [6,13,20,27,4,11,18,25]

# Figuring out F Stat P-val

In [7]:
jan_features = pp.read_npy1('/u/project/cratsch/tescala/month_split_right/features_jan_2015.npy', features=True)
jan_targets = pp.read_npy1('/u/project/cratsch/tescala/month_split_right/targets_jan_2015.npy', targets=True)

In [8]:
z_list = ['z2', 'z3', 'z4', 'z5', 'z6', 'z7', 'z8', 'z9', 'z10', 'z11', 'z12']
new_jan_features = jan_features.loc[:, 'datetime':'z1']
for z in z_list:
    new_jan_features[z] = jan_features[z]

In [9]:
features, mean, std = normalize_and_fill(new_jan_features)

In [38]:
f, p, summary, f_pvalue = wls_results_wald(features, jan_targets, x='x2', z='z8', y='y2')
f_pvalue * 10**350

OverflowError: int too large to convert to float

In [29]:
summary

<class 'statsmodels.iolib.summary.Summary'>
"""
                            WLS Regression Results                            
==============================================================================
Dep. Variable:                     y2   R-squared:                       0.000
Model:                            WLS   Adj. R-squared:                  0.000
Method:                 Least Squares   F-statistic:                     173.6
Date:                Fri, 06 Aug 2021   Prob (F-statistic):          5.30e-149
Time:                        14:54:30   Log-Likelihood:             1.8596e+07
No. Observations:             4050786   AIC:                        -3.719e+07
Df Residuals:                 4050781   BIC:                        -3.719e+07
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const      -2.668e-07   8.96e-07     -0.298      0.766   -2.02e-06    1.49e-06
x1         -3.644e-05   1.85e-06    -19.668      0.000   -4.01e-05   -3.28e-05
x2         -9.816e-06   7.05e-06     -1.392      0.164   -2.36e-05    4.01e-06
x3         -1.644e-05   1.18e-05     -1.397      0.162   -3.95e-05    6.62e-06
x4          6.238e-06   1.68e-06      3.724      0.000    2.95e-06    9.52e-06
==============================================================================
Omnibus:                   586089.500   Durbin-Watson:                   1.992
Prob(Omnibus):                  0.000   Jarque-Bera (JB):          9271081.215
Skew:                           0.019   Prob(JB):                         0.00
Kurtosis:                      10.411   Cond. No.                         13.3
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [26]:
f, p = eda.ols_results(features, jan_targets, x='x13', z='z12', y='y2')
p

2.6240824008836123e-05

In [11]:
p

4.090195924716568e-95

In [12]:
f

111.37985162800378